In [1]:
import cplex
from asignacion_cuadrillas import armar_lp, resolver_lp, mostrar_solucion, cargar_instancia_con_configuracion
from cplex._internal._constants import CPX_VARSEL_DEFAULT, CPX_VARSEL_STRONG, CPX_ON, CPX_OFF
import numpy as np
import os
import psutil


In [2]:
def configurar_y_resolver(path, **config):
    instancia = cargar_instancia_con_configuracion(path, **config)
    prob = cplex.Cplex()
    armar_lp(prob, instancia, shouldWrite=False)
    resolver_lp(prob, instancia)
    mostrar_solucion(prob, instancia)

activar_restriccion_opcional_1 = False, 
activar_restriccion_opcional_2 = False,
tolerancia = 1e-6,
seleccion_nodo = None,
seleccion_variable = None,
heuristica_primal = None,
preproceso = None

In [3]:
configurar_y_resolver("./entradas/entrada4.txt", activar_restriccion_opcional_1 = True, activar_restriccion_opcional_2 = True, preproceso = CPX_OFF, seleccion_variable = CPX_VARSEL_STRONG)

11 [[0, 1]]
1143
[[1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142]]
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10)]


TypeError: 'int' object is not iterable

In [5]:
def configurar_y_resolver_devuelve_tiempos(path, **config):
    instancia = cargar_instancia_con_configuracion(path, **config)
    prob = cplex.Cplex()
    armar_lp(prob, instancia, shouldWrite=False)
    resolver_lp(prob, instancia)
    return instancia.tiempo_de_computo


In [8]:
valores = [0.5,1.5]
tiempos_valores_heuristica = []
for i in valores:
    tiempos_heuristica=[]
    cont = 0
    while cont != 5:
        t = configurar_y_resolver_devuelve_tiempos("./entradas/entrada3.txt", activar_restriccion_opcional_1 = False, preproceso = CPX_OFF, heuristica_primal = i)
        tiempos_heuristica.append(t)
        cont = cont + 1
    tiempo_medio = np.mean(tiempos_heuristica)
    tiempos_valores_heuristica.append(tiempo_medio)

print(valores,tiempos_valores_heuristica)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_MIPGap                   9.9999999999999995e-07
CPXPARAM_MIP_Strategy_HeuristicEffort            0.5
Found incumbent of value 0.000000 after 0.02 sec. (9.54 ticks)
Clique table members: 21814.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 12.89 sec. (2935.19 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000   1.22850e+07              --- 
      0     0   182000.0000    99        0.0000   182000.0000     1862     --- 
*     0+    0                       182000.0000   182000.0000             0.00%
      0     0        cutoff         182000.0000  

In [5]:
tiempos_default=[]
cont = 0
while cont != 5:
    t = configurar_y_resolver_devuelve_tiempos("./entradas/entrada3.txt", activar_restriccion_opcional_1 = False)
    tiempos_default.append(t)
    cont = cont + 1
    

print(np.mean(tiempos_default))

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_MIPGap                   9.9999999999999995e-07
Found incumbent of value 0.000000 after 0.00 sec. (9.54 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 330 rows and 0 columns.
Reduced MIP has 1609794 rows, 13050 columns, and 7129908 nonzeros.
Reduced MIP has 12930 binaries, 120 generals, 0 SOSs, and 0 indicators.
Presolve time = 7.22 sec. (5639.29 ticks)
Probing time = 0.81 sec. (150.47 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1609794 rows, 13050 columns, and 7129908 nonzeros.
Reduced MIP has 12930 binaries, 120 generals, 0 SOSs, and 0 indicators.
Presolve time = 28.28 sec. (11388.68 ticks)
Probing time = 0.84 sec. (141.43 ticks)
Clique table members: 32452.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution ti

In [31]:
tiempos_strong_b=[]
cont = 0
while cont != 5:
    t,m = configurar_y_resolver_devuelve_tiempos("./entradas/entrada3.txt", activar_restriccion_opcional_1 = False, preproceso = CPX_OFF, seleccion_variable = CPX_VARSEL_STRONG)
    tiempos_strong_b.append(t)
    cont = cont + 1
    

print(np.mean(tiempos_strong_b))

Exception ignored in: <function Cplex.__del__ at 0x0000020F3B6B0B80>
Traceback (most recent call last):
  File "c:\Users\sanel\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\__init__.py", line 925, in __del__
    self.end()
  File "c:\Users\sanel\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\__init__.py", line 880, in end
    def end(self):
KeyboardInterrupt: 


Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_MIP_Tolerances_MIPGap                   9.9999999999999995e-07
Found incumbent of value 0.000000 after 0.02 sec. (9.54 ticks)
Clique table members: 21814.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.


In [15]:
tiempos_pseudocosto = []
cont = 0
while cont != 5:
    t = configurar_y_resolver_devuelve_tiempos("./entradas/entrada3.txt", activar_restriccion_opcional_1 = False, preproceso = CPX_OFF, seleccion_variable = 2)
    tiempos_pseudocosto.append(t)
    cont = cont + 1

print(np.mean(tiempos_pseudocosto))

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Strategy_VariableSelect             2
CPXPARAM_MIP_Tolerances_MIPGap                   9.9999999999999995e-07
Found incumbent of value 0.000000 after 0.00 sec. (9.54 ticks)
Clique table members: 21814.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 8.88 sec. (2935.19 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000   1.22850e+07              --- 
      0     0   182000.0000    99        0.0000   182000.0000     1862     --- 
*     0+    0                       182000.0000   182000.0000             0.00%
      0     0        cutoff         182000.0000   18

In [16]:
tiempos_default = []
cont = 0
while cont != 5:
    t = configurar_y_resolver_devuelve_tiempos("./entradas/entrada3.txt", activar_restriccion_opcional_1 = False, preproceso = CPX_OFF, seleccion_variable = CPX_VARSEL_DEFAULT)
    tiempos_default.append(t)
    cont = cont + 1

print(np.mean(tiempos_default))

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_MIPGap                   9.9999999999999995e-07
Found incumbent of value 0.000000 after 0.02 sec. (9.54 ticks)
Clique table members: 21814.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 8.81 sec. (2935.19 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000   1.22850e+07              --- 
      0     0   182000.0000    99        0.0000   182000.0000     1862     --- 
*     0+    0                       182000.0000   182000.0000             0.00%
      0     0        cutoff         182000.0000   182000.0000     1865    0.00%
Elapsed time = 34.12 se

In [18]:
tiempos_deep = []
cont = 0
while cont != 5:
    t = configurar_y_resolver_devuelve_tiempos("./entradas/entrada3.txt", activar_restriccion_opcional_1 = False, preproceso = CPX_OFF, seleccion_variable = CPX_VARSEL_STRONG, seleccion_nodo = 0)
    tiempos_deep.append(t)
    cont = cont + 1

print(np.mean(tiempos_deep))

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_MIP_Tolerances_MIPGap                   9.9999999999999995e-07
Found incumbent of value 0.000000 after 0.02 sec. (9.54 ticks)
Clique table members: 21814.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 9.44 sec. (2935.19 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000   1.22850e+07              --- 
      0     0   182000.0000    99        0.0000   182000.0000     1862     --- 
*     0+    0                       176000.0000   182000.0000             3.41%
*     0     0      integral     0   182000.0000     

In [20]:
tiempos_bestestimate = []
cont = 0
while cont != 5:
    t = configurar_y_resolver_devuelve_tiempos("./entradas/entrada3.txt", activar_restriccion_opcional_1 = False, preproceso = CPX_OFF, seleccion_variable = CPX_VARSEL_STRONG, seleccion_nodo = 2)
    tiempos_bestestimate.append(t)
    cont = cont + 1

print(np.mean(tiempos_bestestimate))

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_MIP_Strategy_Search                     2
CPXPARAM_MIP_Tolerances_MIPGap                   9.9999999999999995e-07
Found incumbent of value 0.000000 after 0.00 sec. (9.54 ticks)
Clique table members: 21814.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 8.53 sec. (2935.19 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000   1.22850e+07              --- 
      0     0   182000.0000    99        0.0000   182000.0000     1862     --- 
*     0+    0                       176000.0000   182000.0000             3.41%
*